In [10]:
!pip install webdriver-manager

In [21]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd

# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(3)

# Naver API key 입력
client_id = 'jOPryrZxBQqmttHYWA6k' 
client_secret = 'SmTcVxIiSD'

# selenium으로 검색 페이지 불러오기 #
naver_urls = []
postdate = []
titles = []

# 검색어 입력
keywords = input("검색할 키워드를 입력해주세요 (쉼표로 구분):")
keyword_list = keywords.split(',')
encText = urllib.parse.quote(' '.join(keyword_list))

# 검색을 끝낼 페이지 입력
end = input("\n크롤링을 끝낼 위치를 입력해주세요. (기본값:1, 최대값:100):")  
if end == "":
    end = 1
else:
    end = int(end)
print("\n 1 ~ ", end, "페이지 까지 크롤링을 진행 합니다")

# 한번에 가져올 페이지 입력
display = input("\n한번에 가져올 페이지 개수를 입력해주세요.(기본값:10, 최대값: 100):")
if display == "":
    display = 10
else:
    display = int(display)
print("\n한번에 가져올 페이지 : ", display, "페이지")

for start in range(1, end * display + 1, display):
    url = f"https://openapi.naver.com/v1/search/blog?query={encText}&start={start}&display={display}"  # JSON 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if rescode == 200:
        response_body = response.read()
        
        data = json.loads(response_body.decode('utf-8'))['items']
        for row in data:
            if 'blog.naver' in row['link']:
                naver_urls.append(row['link'])
                postdate.append(row['postdate'])
                title = row['title']
                # html태그제거
                pattern1 = '<[^>]*>'
                title = re.sub(pattern=pattern1, repl='', string=title)
                titles.append(title)
        time.sleep(2)
    else:
        print("Error Code:" + rescode)

###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

contents = []
comments_texts = []
try:
    for i in naver_urls:
        print(i)
        driver.get(i)
        time.sleep(5)  # 대기시간 변경 가능

        iframe = driver.find_element(By.ID , "mainFrame") # id가 mainFrame이라는 요소를 찾아내고 -> iframe임
        driver.switch_to.frame(iframe) # 이 iframe이 내가 찾고자하는 html을 포함하고 있는 내용

        source = driver.page_source
        html = BeautifulSoup(source, "html.parser")
        
        # 기사 텍스트만 가져오기
        content = html.select("div.se-main-container")
        #  list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')
        content = content.replace('\n', '')
        content = content.replace('\u200b', '')
        contents.append(content)

    news_df = pd.DataFrame({'title': titles, 'content': contents, 'date': postdate})
    news_df.to_csv('blog.csv', index=False, encoding='utf-8-sig')
except:
    contents.append('error')
    news_df = pd.DataFrame({'title': titles, 'content': contents, 'date': postdate})
    news_df.to_csv('blog.csv', index=False, encoding='utf-8-sig')


 1 ~  20 페이지 까지 크롤링을 진행 합니다

한번에 가져올 페이지 :  25 페이지
https://blog.naver.com/kem8732/223711218718
https://blog.naver.com/matrixjh/223707270672
https://blog.naver.com/balll1861/223712747039
https://blog.naver.com/lkh9007/223662821928
https://blog.naver.com/dnjstpdms24/223685144996
https://blog.naver.com/sanaella/223700356363
https://blog.naver.com/rhrh208/223700691857
https://blog.naver.com/0392oe/223679961017
https://blog.naver.com/needtime0514/223697652934
https://blog.naver.com/luk1004/223625558316
https://blog.naver.com/jangsally97/223686853689
https://blog.naver.com/1qa145613/223706601263
https://blog.naver.com/maroyuki/223704202659
https://blog.naver.com/you1620/223692182359
https://blog.naver.com/jinidag/223703266821
https://blog.naver.com/dakhosa/223634741912
https://blog.naver.com/lkh9007/223692894943
https://blog.naver.com/heejin0906/223649067057
https://blog.naver.com/farbemaria/223710234730
https://blog.naver.com/bacuhime/223690137570
https://blog.naver.com/blk_lg_ryan/2237046

In [22]:
import pandas as pd 

df = pd.read_csv('blog500.csv')
df

,title,content,date
0,2024년 12월 성수 팝업 - tvN 홀리데이 파티 (팝업 기간은...,[안녕하세여! 여러분~! 여러분~ 2025년 되었습니다!! 새해 복 많이 받으세...,20250101
1,"[성수] 모푸샌드 팝업 스토어, 12월 마무리는 귀여운...","[ 예삐 학교가 12월 26, 27일을 재량 휴업일로 지정하는 바람에 휴가를 쓸 수...",20241227
2,12월 2주차 주간일기4 - 모푸샌드 mofusand 성수 팝업...,[불과 3년 전까지만 해도 우리 집은한 달에 한두 번은 쇼핑을 하러이마트나 코스트코...,20250102
3,유니클로 팝업 성수 히트텍 요원 되기?,[한때는 불매운동도 하더만 점점 이미지 좋아져서(?)성수 팝업도 오픈했어요하긴 겨울...,20241116
4,12월 성수 팝업 스토어 르쿠르제 팝업 예약없이 방문후기 (ft....,[ 12월 성수 팝업스토어 일정 중 제일 핫한 르쿠르제 팝업 &lt;Unfo...,20241206
...,...,...,...
495,"성수 팝업 핫플 추천, 빼빼로 미니 팝업 이벤트 참여해요!",[ 다양한 팝업스토어 구경하는 재미가 잇는성수동에서 이번에 빼빼로 팝업스토어가 오픈...,20241020
496,"뉴발란스 성수 993 팝업스토어 오픈, 아트갤러리 방문기",[본 포스팅은 ‘뉴발란스’ 로부터 소정의 원고료를 제공받아 솔직하게 작성된 후기입니...,20241017
497,퍼셀 긱 사이언티스트 랩 성수 팝업(1등 당첨 방문후기),[퍼셀을 알게 된건 신세계면세점!퍼셀 XXL 메이크오버 스틱 구매해서 써봤었는데코옆...,20241024
498,"11월 성수 팝업, 핫플 돌다가... 패딩 더현대서울팝업 정보 겟",[본 포스팅은 듀베티카로부터소정의 원고료를 제공받아 작성되었습니다 요즘 특히 핫한 ...,20241110


In [ ]:
# 2022년 이후 데이터만 가져오기
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(3)
# Naver API key 입력
client_id = 'jOPryrZxBQqmttHYWA6k'
client_secret = 'SmTcVxIiSD'
# selenium으로 검색 페이지 불러오기 #
naver_urls = []
postdate = []
titles = []
# 검색어 입력
keywords = input("검색할 키워드를 입력해주세요 (쉼표로 구분):")
keyword_list = keywords.split(',')
encText = urllib.parse.quote(' '.join(keyword_list))
# 검색을 끝낼 페이지 입력
end = input("\n크롤링을 끝낼 위치를 입력해주세요. (기본값:1, 최대값:100):")
if end == "":
    end = 1
else:
    end = int(end)
print("\n 1 ~ ", end, "페이지 까지 크롤링을 진행 합니다")
# 한번에 가져올 페이지 입력
display = input("\n한번에 가져올 페이지 개수를 입력해주세요.(기본값:10, 최대값: 100):")
if display == "":
    display = 10
else:
    display = int(display)
print("\n한번에 가져올 페이지 : ", display, "페이지")
for start in range(1, end * display + 1, display):
    url = f"https://openapi.naver.com/v1/search/blog?query={encText}&start={start}&display={display}&sort=date"  # JSON 결과, 최신순 정렬
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if rescode == 200:
        response_body = response.read()
        data = json.loads(response_body.decode('utf-8'))['items']
        for row in data:
            post_date = datetime.strptime(row['postdate'], '%Y%m%d')
            if post_date.year >= 2022:
                if 'blog.naver' in row['link']:
                    naver_urls.append(row['link'])
                    postdate.append(row['postdate'])
                    title = row['title']
                    # html태그제거
                    pattern1 = '<[^>]*>'
                    title = re.sub(pattern=pattern1, repl='', string=title)
                    titles.append(title)
        time.sleep(2)
    else:
        print("Error Code:" + rescode)
###naver 기사 본문 및 제목 가져오기###
# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
contents = []
comments_texts = []
try:
    for i in naver_urls:
        print(i)
        driver.get(i)
        time.sleep(5)  # 대기시간 변경 가능
        iframe = driver.find_element(By.ID , "mainFrame") # id가 mainFrame이라는 요소를 찾아내고 -> iframe임
        driver.switch_to.frame(iframe) # 이 iframe이 내가 찾고자하는 html을 포함하고 있는 내용
        source = driver.page_source
        html = BeautifulSoup(source, "html.parser")
        # 기사 텍스트만 가져오기
        content = html.select("div.se-main-container")
        #  list합치기
        content = ''.join(str(content))
        # html태그제거 및 텍스트 다듬기
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')
        content = content.replace('\n', '')
        content = content.replace('\u200b', '')
        contents.append(content)
    news_df = pd.DataFrame({'title': titles, 'content': contents, 'date': postdate})
    news_df.to_csv('blog.csv', index=False, encoding='utf-8-sig')
except:
    contents.append('error')
    news_df = pd.DataFrame({'title': titles, 'content': contents, 'date': postdate})
    news_df.to_csv('blog.csv', index=False, encoding='utf-8-sig')

In [77]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime

# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(3)

# Naver API key
client_id = 'jOPryrZxBQqmttHYWA6k' 
client_secret = 'SmTcVxIiSD'

# 데이터 저장용 리스트
naver_urls = []
postdate = []
titles = []
contents = []

# 사용자 입력
keywords = input("검색할 키워드를 입력해주세요 (쉼표로 구분):")
keyword_list = keywords.split(',')
encText = urllib.parse.quote(' '.join(keyword_list))

end = input("\n크롤링을 끝낼 위치를 입력해주세요 (기본값:1, 최대값:100):")
end = 1 if end == "" else int(end)
print(f"\n1 ~ {end} 페이지까지 크롤링을 진행합니다")

display = input("\n한번에 가져올 페이지 개수를 입력해주세요 (기본값:10, 최대값:100):")
display = 10 if display == "" else int(display)
print(f"\n한번에 가져올 페이지: {display} 페이지")

# 검색 기간 설정 (2024년)
dfrom = "20240101"
dto = "20240103"

try:
    # 블로그 검색 API 호출
    for start in range(1, end * display + 1, display):
        url = f"https://openapi.naver.com/v1/search/blog?query={encText}&start={start}&display={display}&sort=date&dfrom={dfrom}&dto={dto}"
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        
        try:
            response = urllib.request.urlopen(request)
            if response.getcode() == 200:
                data = json.loads(response.read().decode('utf-8'))
                
                print(f"\n=== 검색 결과 ===")
                print(f"검색 기간: 2024년 1월 1일 ~ 1월 3일")
                print(f"총 게시물: {data.get('total', 0)}개")
                print(f"현재 페이지: {start//display + 1}")
                print(f"페이지 내 게시물: {len(data.get('items', []))}개")
                print("==================\n")
                
                for item in data.get('items', []):
                    if 'blog.naver' in item['link']:
                        naver_urls.append(item['link'])
                        postdate.append(item['postdate'])
                        titles.append(re.sub('<[^>]*>', '', item['title']))
                        
                print(f"현재까지 수집된 게시물: {len(titles)}개")
                time.sleep(2)
                
        except Exception as e:
            print(f"API 요청 중 오류: {e}")
            continue
    
    # 블로그 본문 크롤링
    for i in naver_urls:
        try:
            print(f"URL 처리 중: {i}")
            driver.get(i)
            time.sleep(3)
            
            iframe = driver.find_element(By.ID, "mainFrame")
            driver.switch_to.frame(iframe)
            
            html = BeautifulSoup(driver.page_source, "html.parser")
            content = html.select("div.se-main-container")
            
            if content:
                content = re.sub('<[^>]*>', '', ''.join(str(content)))
                content = content.replace('\n', ' ').replace('\u200b', '')
            else:
                content = 'content not found'
                
            contents.append(content)
            
        except Exception as e:
            print(f"컨텐츠 수집 중 오류: {e}")
            contents.append('error')
            
    # 결과 저장
    min_length = min(len(titles), len(contents), len(postdate))
    news_df = pd.DataFrame({
        'title': titles[:min_length],
        'content': contents[:min_length],
        'date': postdate[:min_length]
    })
    news_df.to_csv('blog_2024.csv', index=False, encoding='utf-8-sig')
    print("크롤링 완료!")
    
except Exception as e:
    print(f"전체 처리 중 오류 발생: {e}")
finally:
    driver.quit()


1 ~ 1 페이지까지 크롤링을 진행합니다

한번에 가져올 페이지: 4 페이지

=== 검색 결과 ===
검색 기간: 2024년 1월 1일 ~ 1월 3일
총 게시물: 212362개
현재 페이지: 1
페이지 내 게시물: 4개

현재까지 수집된 게시물: 4개
URL 처리 중: https://blog.naver.com/marchone31/223713926845
URL 처리 중: https://blog.naver.com/gogomi123_/223713915387
URL 처리 중: https://blog.naver.com/duswl0876/223713909880
URL 처리 중: https://blog.naver.com/jsh3321/223713906800
크롤링 완료!
